- 思路
    - 学习 cv2 预处理方式
    - 对单张图片预处理
    - 文件夹内存所有图片, 批量预处理
    - 显示处理进度

- 原则是在尽可能不丢失信息的情况下, 最大不同
- 预处理方案
    - 原始图片缩放至 299×299
    - 左右旋转 20°
    - 向下平移 0.2
    - 水平翻转
    - 在水平翻转基础上左右旋转 20°
    - 向下平移 0.2

> 18.5.11
* 添加高斯噪声

# 单体变换

## 放射变换

In [1]:
import cv2
import numpy as np

In [55]:
img = cv2.imread('image.jpg')

In [56]:
# 沿着横纵轴放大2倍，然后平移(-20,-20)，最后沿原图大小截取，等效于裁剪并放大
M_crop_trans = np.array([
    [1, 0, -20],
    [0, 1, -20]
], dtype=np.float32)

img_crop_trans = cv2.warpAffine(img, M_crop_trans, (512, 512))
cv2.imwrite('img_crop_trans.jpg', img_crop_trans)

True

In [57]:
# x轴的剪切shear变换，角度45°
theta = 45 * np.pi / 180
M_shear = np.array([
    [1, np.tan(theta), 0],
    [0, 1, 0]
], dtype=np.float32)

img_sheared = cv2.warpAffine(img, M_shear, (512, 512))
cv2.imwrite('img_sheared.jpg', img_sheared)

True

In [58]:
# 顺时针旋转，角度45°
theta = 45 * np.pi / 180
M_rotate = np.array([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0]
], dtype=np.float32)

img_rotated = cv2.warpAffine(img, M_rotate, (512, 512))
cv2.imwrite('img_rotated.jpg', img_rotated)

True

In [59]:
# 某种变换，具体旋转+缩放+旋转组合可以通过SVD分解理解
M = np.array([
    [1, 1.5, -400],
    [0.5, 2, -100]
], dtype=np.float32)

img_transformed = cv2.warpAffine(img, M, (512, 512))
cv2.imwrite('img_transformed.jpg', img_transformed)

True

## 缩放scale

In [60]:
import cv2

# 读取一张照片
img = cv2.imread('image.jpg')

# 缩放成200x200的图像
img_512x512 = cv2.resize(img, (512, 512), interpolation=cv2.INTER_CUBIC)
img_299x299 = cv2.resize(img, (299, 299), interpolation=cv2.INTER_CUBIC)

cv2.imwrite('img_512x512.jpg', img_512x512)
cv2.imwrite('img_299x299.jpg', img_299x299)

True

## 旋转rotate

In [61]:
img = cv2.imread('image.jpg')
rows,cols,channel = img.shape
#90度旋转
M = cv2.getRotationMatrix2D((cols/2,rows/2),0,1.05)
dst = cv2.warpAffine(img,M,(cols,rows))

cv2.imwrite('image_rotate_10.jpg', dst)

True

## 裁剪crop

In [62]:
img = cv2.imread('image.jpg')

#裁剪
crop_img = img[0:-10, 0:-10]
cv2.imwrite('crop_img.jpg', crop_img)

True

## 填充pad

In [63]:
# 在上张图片的基础上，上下各填充50像素，填充值为128，生成新的的图像
pad_img = cv2.copyMakeBorder(img, 50, 50, 0, 0, cv2.BORDER_REPLICATE)
cv2.imwrite('pad_img.jpg', pad_img)

True

In [3]:
## 图像填充
img = cv2.imread('image.jpg')
row, column, channels = img.shape
max_value = max(row, column)
pad_img= cv2.copyMakeBorder(img,(max_value-row)//2,(max_value-row)//2,(max_value-column)//2,(max_value-column)//2,cv2.BORDER_CONSTANT)

img_299x299 = cv2.resize(pad_img, (299, 299), interpolation=cv2.INTER_AREA)
cv2.imwrite('img_299x299.jpg', img_299x299)

## 平移translate

In [66]:
img = cv2.imread('image.jpg')
rows,cols,channel = img.shape
# 平移矩阵M：[[1,0,x],[0,1,y]]
M = np.float32([[1,0,0],[0,1,int(cols*0.2)]])
dst = cv2.warpAffine(img,M,(cols,rows))
cv2.imwrite('translate_img.jpg', dst)

True

## 翻转flip

In [67]:
# 水平翻转
flip_horiz_img = cv2.flip(img, 1)
# 垂直翻转
flip_verti_img = cv2.flip(img, 0)
# 水平垂直翻转
flip_horandver_img = cv2.flip(img, -1)
cv2.imwrite('flip_horiz_img.jpg', flip_horiz_img)

True

## 高斯噪声

In [68]:
import cv2
import numpy as np
peppers = cv2.imread("image.jpg")
row, column, channels = peppers.shape

In [69]:
peppers.astype("float")
Gauss_noise = np.random.normal(0, 50, (row, column, channels))
Gauss = peppers + Gauss_noise
Gauss = np.where(Gauss < 0, 0, np.where(Gauss > 255, 255, Gauss))

In [70]:
cv2.imwrite('Gauss.jpg', Gauss.astype("uint8"))

True

## 色彩抖动

In [6]:
from PIL import Image, ImageEnhance, ImageOps, ImageFile

In [ ]:
import cv2
import numpy as np
iamge = 'image.jpg'
Image.open(image, mode="r")

In [10]:
image = 'image.jpg'
img = Image.open(image, mode="r")

In [15]:
def randomColor(image):
    """
    对图像进行颜色抖动
    :param image: PIL的图像image
    :return: 有颜色色差的图像image
    """
    random_factor = np.random.randint(0, 31) / 10.  # 随机因子
    color_image = ImageEnhance.Color(image).enhance(random_factor)  # 调整图像的饱和度
    random_factor = np.random.randint(10, 21) / 10.  # 随机因子
    brightness_image = ImageEnhance.Brightness(color_image).enhance(random_factor)  # 调整图像的亮度
    random_factor = np.random.randint(10, 21) / 10.  # 随机因1子
    contrast_image = ImageEnhance.Contrast(brightness_image).enhance(random_factor)  # 调整图像对比度
    random_factor = np.random.randint(0, 31) / 10.  # 随机因子
    return ImageEnhance.Sharpness(contrast_image).enhance(random_factor)  # 调整图像锐度

In [16]:
img_collar = randomColor(img)

In [18]:
newname = 'img_collar.jpg'
img_collar.save(newname)

## 直方图

In [2]:
import cv2
import numpy as np
from skimage import data, img_as_float
from skimage import exposure

In [8]:
img = cv2.imread("image.jpg")

In [9]:
out = exposure.equalize_hist(img)*255

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/exposure/exposure.py:63: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  warn("This might be a color image. The histogram will be "


In [10]:
cv2.imwrite('img_eqh.jpg', out)

True

## 对比拉伸

In [11]:
p2, p98 = np.percentile(img, (2, 98))
out = exposure.rescale_intensity(img, in_range=(p2, p98))

In [12]:
cv2.imwrite('img_cs.jpg', out)

True

# 单张图片预处理

In [71]:
# 左右旋转 20°
img = cv2.imread('image.jpg')

img = cv2.imread(currentPath)
rows,cols, channels = img.shape
max_value = max(rows, cols)
pad_img = cv2.copyMakeBorder(img,(max_value-rows)//2,(max_value-rows)//2,(max_value-cols)//2,(max_value-cols)//2,cv2.BORDER_CONSTANT)
dst0 = cv2.resize(pad_img, (299, 299), interpolation=cv2.INTER_AREA)

M1 = cv2.getRotationMatrix2D((cols/2,rows/2),20,1)
M2 = cv2.getRotationMatrix2D((cols/2,rows/2),-20,1)
dst1 = cv2.warpAffine(img_resize,M1,(cols,rows))
dst2 = cv2.warpAffine(img_resize,M2,(cols,rows))

cv2.imwrite('img_resize.jpg', img_resize)
cv2.imwrite('./preprocessing/image_rotate_20.jpg', dst1)
cv2.imwrite('./preprocessing/image_rotate_-20.jpg', dst2)

True

In [72]:
# 向下平移 0.2
M3 = np.float32([[1,0,0],[0,1,int(cols*0.2)]])
dst3 = cv2.warpAffine(img_resize,M3,(cols,rows))
cv2.imwrite('./preprocessing/translate_img.jpg', dst3)

True

In [80]:
# 水平翻转
flip_horiz_img = cv2.flip(img_resize, 1)
cv2.imwrite('./preprocessing/flip_horiz_img.jpg', flip_horiz_img)

True

In [81]:
# 对翻转后图片, 左右旋转 20°
M4 = cv2.getRotationMatrix2D((cols/2,rows/2),20,1)
M5 = cv2.getRotationMatrix2D((cols/2,rows/2),-20,1)
dst4 = cv2.warpAffine(flip_horiz_img,M4,(cols,rows))
dst5 = cv2.warpAffine(flip_horiz_img,M5,(cols,rows))

cv2.imwrite('./preprocessing/image_flip_rotate_20.jpg', dst4)
cv2.imwrite('./preprocessing/image_flip_rotate_-20.jpg', dst5)

True

In [82]:
# 向下平移 0.2
M6 = np.float32([[1,0,0],[0,1,int(cols*0.2)]])
dst6 = cv2.warpAffine(flip_horiz_img,M6,(cols,rows))
cv2.imwrite('./preprocessing/translate_flip_img.jpg', dst6)

True

# 批量预处理

In [4]:
import os
import os.path

In [11]:
rootdir = './train/collar_design_labels'  # 指明被遍历的文件夹
m = 0
outdir = '/Users/hugo/Projects/bitbucket/fashionai/ipynb/LiChuan/preprocessing'

for parent, dirnames, filenames in os.walk(rootdir):
    for filename in filenames:
        currentPath = os.path.join(parent, filename)

        # 先填充, 再缩放
        img = cv2.imread(currentPath)
        rows,cols,channels = img.shape
        max_value = max(rows, cols)
        pad_img = cv2.copyMakeBorder(img,(max_value-rows)//2,(max_value-rows)//2,(max_value-cols)//2,(max_value-cols)//2,cv2.BORDER_CONSTANT)
        dst0 = cv2.resize(pad_img, (299, 299), interpolation=cv2.INTER_AREA)

        # 顺时针旋转 +-20°
        M1 = cv2.getRotationMatrix2D((299/2,299/2),20,1) 
        M2 = cv2.getRotationMatrix2D((299/2,299/2),-20,1) 
        dst4 = cv2.warpAffine(dst0,M1,(299,299))
        dst5 = cv2.warpAffine(dst0,M2,(299,299))
        
#         # 向下平移 0.2
#         M3 = np.float32([[1,0,0],[0,1,int(cols*0.2)]])
#         dst3 = cv2.warpAffine(dst0,M3,(cols,rows))
        
        # 水平翻转
        dst1 = cv2.flip(dst0, 1)
        
#         # 高斯噪声
#         dst0.astype("float")
#         Gauss_noise = np.random.normal(0, 20, (rows, cols, channels))
#         dst2 = dst0 + Gauss_noise
#         dst2 = np.where(dst2 < 0, 0, np.where(dst2 > 255, 255, dst2))
        
#         # 直方图
#         dst2 = exposure.equalize_hist(dst1)*255
        
#         # 对比拉伸
#         p2, p98 = np.percentile(dst0, (2, 98))
#         dst3 = exposure.rescale_intensity(dst0, in_range=(p2, p98))
        
        # 对比拉伸
        p2, p98 = np.percentile(dst1, (2, 98))
        dst6 = exposure.rescale_intensity(dst1, in_range=(p2, p98))
        
#         cv2.imwrite(outdir + filename + "-0.jpg", dst0)
#         cv2.imwrite(outdir + filename + "-1.jpg", dst1)
#         cv2.imwrite(outdir + filename + "-2.jpg", dst2)
#         cv2.imwrite(outdir + filename + "-3.jpg", dst3)
        cv2.imwrite(outdir + filename + "-4.jpg", dst0)
        cv2.imwrite(outdir + filename + "-5.jpg", dst4)
        cv2.imwrite(outdir + filename + "-6.jpg", dst5)
        cv2.imwrite(outdir + filename + "-7.jpg", dst6)
        m += 1
        if m%2 == 0:
            print('{} images have preprocessed.'.format(m))

2 images have preprocessed.
4 images have preprocessed.
6 images have preprocessed.
8 images have preprocessed.
10 images have preprocessed.


## np.array 的合并

In [21]:
X_train = np.zeros((8, 299, 299), dtype=np.uint8)
y_train = np.zeros((8, 9), dtype=np.uint8)

In [22]:
X_valid = np.zeros((2, 299, 299), dtype=np.uint8)
y_valid = np.zeros((2, 9), dtype=np.uint8)

In [24]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((8, 299, 299), (8, 9), (2, 299, 299), (2, 9))

In [29]:
X_comb = np.concatenate((X_train, X_valid), axis=0)
y_comb = np.concatenate((y_train, y_valid), axis=0)

In [28]:
X_comb.shape, y_comb.shape

(10, 299, 299)